In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import date,datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date,datetime,timedelta
import time
from dbase import Database

In [ ]:
db = Database('emtia_db.db')

def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]


url = "https://www.marketwatch.com/investing/future/mtfc00"


chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument( "user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2919.83 Safari/537.36")

driver = webdriver.Chrome(options = chrome_options)

count = 0
while count < 200:
    try:
        driver.get(url)

        html_content = driver.page_source

        soup = BeautifulSoup(html_content, "html.parser")

        brent_price = [item.text for item in soup.find_all('div', attrs={"class": "element element--table overflow--table FuturesContracts"})[0].find_all('td',attrs = {"class":"table__cell"})]

        prices = list(divide_chunks(brent_price,7))

        #for price in prices:
        #    db.insert_brent(price[0], float(str(price[1]).split('$')[1]), price[2], float(str(price[3]).split('$')[1]), float(str(price[4]).split('$')[1]), float(str(price[5]).split('$')[1]), datetime.now(),url)

        brent_name = [item[0] for item in prices]
        brent_last = [item[1] for item in prices]
        brent_chg = [item[2] for item in prices]
        brent_open = [item[3] for item in prices]
        brent_high = [item[4] for item in prices]
        brent_low = [item[5] for item in prices]
        brent_time = datetime.now()

        brent_future = {"Names": brent_name,
                        "Price_last": brent_last,
                        "Price_chg": brent_chg,
                        "Price_open": brent_open,
                        "Price_high": brent_high,
                        "Price_low": brent_low,
                        "Time": brent_time}

        brent = pd.DataFrame()

        brent["names"] = brent_future["Names"]
        brent["price_last"] = brent_future["Price_last"]
        brent["price_chg"] = brent_future["Price_chg"]
        brent["price_open"] = brent_future["Price_open"]
        brent["price_high"] = brent_future["Price_high"]
        brent["price_low"] = brent_future["Price_low"]
        brent["price_time"] = brent_future["Time"]

        print("DONE")
        print(brent)
    except:
        print("ERROR")
    count += 1
    time.sleep(900)

DONE
                                            names price_last price_chg  \
0  Coal (API2) CIF ARA (ARGUS-McCloskey) Jun 2022    $324.50      0.00   
1  Coal (API2) CIF ARA (ARGUS-McCloskey) Jul 2022    $291.50      0.00   
2  Coal (API2) CIF ARA (ARGUS-McCloskey) Aug 2022    $277.65      0.00   
3  Coal (API2) CIF ARA (ARGUS-McCloskey) Sep 2022    $270.95      0.00   
4  Coal (API2) CIF ARA (ARGUS-McCloskey) Oct 2022    $264.40      0.00   
5  Coal (API2) CIF ARA (ARGUS-McCloskey) Nov 2022    $258.00      0.00   
6  Coal (API2) CIF ARA (ARGUS-McCloskey) Dec 2022    $252.40      0.00   
7  Coal (API2) CIF ARA (ARGUS-McCloskey) Jan 2023    $246.95      0.00   
8  Coal (API2) CIF ARA (ARGUS-McCloskey) Feb 2023    $241.60      0.00   
9  Coal (API2) CIF ARA (ARGUS-McCloskey) Mar 2023    $235.55      0.00   

  price_open price_high price_low                 price_time  
0      $0.00      $0.00     $0.00 2022-06-09 14:56:21.718434  
1      $0.00      $0.00     $0.00 2022-06-09 14:56:21.

# BARCHART

In [4]:
db = Database('emtia_db.db')
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument( "user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2919.83 Safari/537.36")


products_all = []


this_year = (date.today()).strftime("%y")
next_year = (date.today()+timedelta(days=365)).strftime("%y")
year_after_next = (date.today()+timedelta(days=730)).strftime("%y")
next_month = (date.today()+timedelta(days=30)).strftime('%b')

products = {"Jan": "ITFF",
          "Feb": "ITFG",
          "Mar": "ITFH",
          "Apr": "ITFJ",
          "May": "ITFK",
          "Jun": "ITFM",
          "Jul": "ITFN",
          "Aug": "ITFQ",
          "Sep": "ITFU",
          "Oct": "ITFV",
          "Nov": "ITFX",
          "Dec": "ITFZ"}

start = date.today().month
left= 13-start+1
count = 0
months = []

start = date.today().month
start_month = date.today()
print(start)
left= 13-start
count =0
months = []
while count<left:
    days_add = 30*count
    month = (start_month+timedelta(days=days_add)).strftime("%b")
    print(month)
    if month == next_month:
        products_all.append(str(products.get(month))+str(this_year))
    else:
        products_all.append(str(products.get(month))+str(this_year))
    count+=1

for key,value in products.items():
    products_all.append(str(products.get(key))+str(next_year))
print(products_all)

TURN = 0
while TURN<200:
    start_time=time.time()
    for item in products_all:
        url = f"https://www.barchart.com/futures/quotes/{item}/overview"
        print(url)
        driver = webdriver.Chrome(options = chrome_options)
        driver.get(url)



        phigh = driver.find_element(by = By.CSS_SELECTOR,value = '#main-content-column > div > div.bc-quote-overview.row.ng-scope > div.small-12.large-5.column > div.bc-quote-row-chart > div.row > div.small-6.column.text-right > div:nth-child(3)')
        plow = driver.find_element(by = By.CSS_SELECTOR,value = '#main-content-column > div > div.bc-quote-overview.row.ng-scope > div.small-12.large-5.column > div.bc-quote-row-chart > div.row > div:nth-child(1) > div:nth-child(3)')
        pname = driver.find_element(by = By.CSS_SELECTOR,value = '#main-content-column > div > div.page-title.symbol-header-info.ng-scope > div.symbol-name > h1 > span.symbol')
        plast = driver.find_element(by = By.CSS_SELECTOR,value = '#main-content-column > div > div.page-title.symbol-header-info.ng-scope > div:nth-child(2) > span.last-change.ng-binding')
        try:
            popen = driver.find_element(by = By.CSS_SELECTOR,value = '#main-content-column > div > div.bc-quote-overview.row.ng-scope > div.small-12.large-5.column > div.bc-quote-row-chart > div.row-chart.ng-isolate-scope.marking > div.mark.ng-scope > span')
            price_open = float(popen.get_attribute("innerText").split(" ")[1])
        except NoSuchElementException:
            try:
                try:
                    popen = driver.find_element(by = By.CSS_SELECTOR,value = '#main-content-column > div > div.bc-quote-overview.row.ng-scope > div.small-12.large-5.column > div.bc-quote-row-chart > div.row-chart.ng-isolate-scope > div.mark.ng-scope > span')
                    price_open = float(popen.get_attribute("innerText").split(" ")[1])
                except ValueError:
                    price_open = ""
            except NoSuchElementException:
                price_open = ""

        try:
            pchg = driver.find_element(by = By.CSS_SELECTOR,value = '#main-content-column > div > div.page-title.symbol-header-info.ng-scope > div:nth-child(2) > span.down > span.last-change.ng-binding')
            price_chg = float(pchg.get_attribute("innerText"))
        except NoSuchElementException:
            try:
                pchg = driver.find_element(by = By.CSS_SELECTOR,value = '#main-content-column > div > div.page-title.symbol-header-info.ng-scope > div:nth-child(2) > span.up > span.last-change.ng-binding')
                price_chg = float(pchg.get_attribute("innerText"))
            except:
                price_chg=""
            


        price_name = pname.get_attribute("innerText")
        try:
            price_last = float(plast.get_attribute("innerText").split(" ")[0])
        except ValueError:
            price_last = float(plast.get_attribute("innerText").split(" ")[0].split('s')[0])
        price_high = float(phigh.get_attribute("innerText"))
        price_low = float(plow.get_attribute("innerText"))


        print(f"PRODUCT:{price_name}\nPrice Last: {price_last}\nPrice Change: {price_chg}\nPrice Open: {price_open}\nPrice High: {price_high}\nPrie Low: {price_low}")
        db.insert_api2(price_name,price_last,price_chg,price_open,price_high,price_low,datetime.now(),url)
        print("Data recorded to database!")
        driver.quit()
    TURN += 1
    final_time = (time.time() - start_time) / 60
    print("Runtime:--- %s minutes ---" % final_time)

6
Jun
Jul
Aug
Sep
Oct
Nov
Dec
['ITFM22', 'ITFN22', 'ITFQ22', 'ITFU22', 'ITFV22', 'ITFX22', 'ITFZ22', 'ITFF23', 'ITFG23', 'ITFH23', 'ITFJ23', 'ITFK23', 'ITFM23', 'ITFN23', 'ITFQ23', 'ITFU23', 'ITFV23', 'ITFX23', 'ITFZ23']
https://www.barchart.com/futures/quotes/ITFM22/overview
PRODUCT:Coal API 2 Cif ARA Argus/McCloskey Swap Futures Jun '22
Price Last: 317.5
Price Change: -4.15
Price Open: 
Price High: 317.5
Prie Low: 317.5
Data recorded to database!
https://www.barchart.com/futures/quotes/ITFN22/overview
PRODUCT:Coal API 2 Cif ARA Argus/McCloskey Swap Futures Jul '22
Price Last: 281.5
Price Change: -8.5
Price Open: 
Price High: 281.5
Prie Low: 281.5
Data recorded to database!
https://www.barchart.com/futures/quotes/ITFQ22/overview
PRODUCT:Coal API 2 Cif ARA Argus/McCloskey Swap Futures Aug '22
Price Last: 270.1
Price Change: -7.4
Price Open: 
Price High: 270.1
Prie Low: 270.1
Data recorded to database!
https://www.barchart.com/futures/quotes/ITFU22/overview
PRODUCT:Coal API 2 Cif ARA Ar

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#main-content-column > div > div.bc-quote-overview.row.ng-scope > div.small-12.large-5.column > div.bc-quote-row-chart > div.row > div.small-6.column.text-right > div:nth-child(3)"}
  (Session info: headless chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x0072D953+2414931]
	Ordinal0 [0x006BF5E1+1963489]
	Ordinal0 [0x005AC6B8+837304]
	Ordinal0 [0x005D9500+1021184]
	Ordinal0 [0x005D979B+1021851]
	Ordinal0 [0x00606502+1205506]
	Ordinal0 [0x005F44E4+1131748]
	Ordinal0 [0x00604812+1198098]
	Ordinal0 [0x005F42B6+1131190]
	Ordinal0 [0x005CE860+976992]
	Ordinal0 [0x005CF756+980822]
	GetHandleVerifier [0x0099CC62+2510274]
	GetHandleVerifier [0x0098F760+2455744]
	GetHandleVerifier [0x007BEABA+551962]
	GetHandleVerifier [0x007BD916+547446]
	Ordinal0 [0x006C5F3B+1990459]
	Ordinal0 [0x006CA898+2009240]
	Ordinal0 [0x006CA985+2009477]
	Ordinal0 [0x006D3AD1+2046673]
	BaseThreadIni

# ICE

In [8]:
db = Database('emtia_db.db')
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument( "user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2919.83 Safari/537.36")
url_ice = f"https://www.theice.com/products/243/API2-Rotterdam-Coal-Futures/data"
driver_ice = webdriver.Chrome(options = chrome_options)
driver_ice.get(url_ice)
time.sleep(5)
start = date.today().month+1
start_month = date.today()+timedelta(days=30)
print(start)
left= 13-start
print(left)
rng = left+12
count = driver_ice.find_elements(by = By.XPATH,value = f'/html/body/div[1]/div/div[1]/div[3]/div[1]/div/div/div[4]/div/div/div[1]/table/tbody')
for i in range(1,len(count)+1):
    try:
        item = driver_ice.find_element(by = By.XPATH,value = f'/html/body/div[1]/div/div[1]/div[3]/div[1]/div/div/div[4]/div/div/div[1]/table/tbody[1]/tr/td[1]')
        last = driver_ice.find_element(by = By.XPATH,value = f'/html/body/div[1]/div/div[1]/div[3]/div[1]/div/div/div[4]/div/div/div[1]/table/tbody[1]/tr/td[2]')
        change = driver_ice.find_element(by = By.XPATH,value = f'/html/body/div[1]/div/div[1]/div[3]/div[1]/div/div/div[4]/div/div/div[1]/table/tbody[1]/tr/td[4]')
        api2_prd = item.get_attribute("innerText")
        print(item)
        try:
            api2_lst = float(last.get_attribute("innerText"))
        except ValueError:
            api2_lst = ""
        try:
            api2_chg = float(change.get_attribute("innerText"))
        except ValueError:
            api2_chg = ""
        print(f"PRODUCT:{api2_prd}\nPrice Last: {api2_lst}\nPrice Change: {api2_chg}")
        db.insert_api2(str("ice_"+str(api2_prd)),api2_lst,api2_chg,"","","",datetime.now(),url_ice)
        time.sleep(0.02)
    except AttributeError:
        continue

10
3
<selenium.webdriver.remote.webelement.WebElement (session="0df71ff0c94f6d091a80d53b826d20ee", element="8661bc91-9afc-424f-b7b1-4ec31e62fa40")>
PRODUCT: JAN23
Price Last: 
Price Change: 0.0
<selenium.webdriver.remote.webelement.WebElement (session="0df71ff0c94f6d091a80d53b826d20ee", element="8661bc91-9afc-424f-b7b1-4ec31e62fa40")>
PRODUCT: JAN23
Price Last: 
Price Change: 0.0
<selenium.webdriver.remote.webelement.WebElement (session="0df71ff0c94f6d091a80d53b826d20ee", element="8661bc91-9afc-424f-b7b1-4ec31e62fa40")>
PRODUCT: JAN23
Price Last: 
Price Change: 0.0
<selenium.webdriver.remote.webelement.WebElement (session="0df71ff0c94f6d091a80d53b826d20ee", element="8661bc91-9afc-424f-b7b1-4ec31e62fa40")>
PRODUCT: JAN23
Price Last: 
Price Change: 0.0
<selenium.webdriver.remote.webelement.WebElement (session="0df71ff0c94f6d091a80d53b826d20ee", element="8661bc91-9afc-424f-b7b1-4ec31e62fa40")>
PRODUCT: JAN23
Price Last: 
Price Change: 0.0
<selenium.webdriver.remote.webelement.WebElement (

In [5]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument( "user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2919.83 Safari/537.36")
url_ice = f"https://www.theice.com/products/243/API2-Rotterdam-Coal-Futures/data"
driver_ice = webdriver.Chrome(options = chrome_options)
driver_ice.get(url_ice)
time.sleep(5)
item = driver_ice.find_element(by = By.XPATH,value = f' /html/body/div[1]/div/div[1]/div[3]/div[1]/div/div/div[4]/div/div/div[1]/table/tbody[1]/tr/td[1]')
                                                        #/html/body/div[1]/div/div[1]/div[3]/div[1]/div/div/div[4]/div/div/div[1]/table/tbody[1]/tr[1]/td[1]/text()
                                                        #/html/body/div[1]/div/div[1]/div[3]/div[1]/div/div/div[4]/div/div/div[1]/table/tbody[1]/tr[1]/td[1]
print(item.get_attribute("innerText"))

 JAN23
